import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import style
import os
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt



import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

print (Bankdata.head())

term_deposits = Bankdata.copy()


Bankdata.dtypes

Bankdata.isnull()
Bankdata.isnull().any()

# Let's see how the numeric data is distributed.
import matplotlib.pyplot as plt

Bankdata.hist(bins=10, figsize=(14,10), color='#E14906')
plt.show()

# Convert the columns that contain a Yes or No. (Binary Columns)
def convert_to_int(Bankdata, new_column, target_column):
    Bankdata[new_column] = Bankdata[target_column].apply(lambda x: 0 if x == 'no' else 1)
    return Bankdata[new_column].value_counts()

convert_to_int(Bankdata, "y_int", "y") #Create a y int
convert_to_int(Bankdata, "housing_int", "housing") # Create housingint column
convert_to_int(Bankdata, "loan_int", "loan") #Create a loan_int column
convert_to_int(Bankdata, "default_int", "default") #Create a default_int column

# Drop the non-binary columns and leave the same column in the form of integers 0 = No and 1 = Yes
Bankdata.drop(['housing', 'loan', 'default','y'], axis=1, inplace=True)

Bankdata.rename(columns={'y_int':'term_deposit'}, inplace=True)


print(Bankdata.head())

dep = Bankdata['term_deposit']
Bankdata.drop(labels=['term_deposit'], axis=1,inplace = True)
Bankdata.insert(0, 'term_deposit', dep)

# Convert duration to minutes of conversation.
decimal_points = 2
Bankdata['duration'] = Bankdata['duration'] / 60
Bankdata['duration'] = Bankdata['duration'].apply(lambda x: round(x, decimal_points))
Bankdata.head()

Bankdata = pd.get_dummies(Bankdata, columns=['job'])


Bankdata.head()

Bankdata.info()

Bankdata = pd.get_dummies(Bankdata, columns=['education'])
Bankdata.head()

Bankdata = pd.get_dummies(Bankdata, columns=['marital','contact','poutcome','month','day_of_week'])
Bankdata.head()

Bankdata.head()

from sklearn.model_selection import train_test_split


#Xs = onpData.drop(onpData.columns[-1], axis = 1)
Xs = Bankdata.drop('term_deposit', axis = 1)

#y = onpData.drop(onpData.columns[:-1], axis = 1)
y = Bankdata['term_deposit']
y.head()

Xs.head()

xTrain, xVald, yTrain, yVald = train_test_split(Xs, y, train_size=0.60, random_state = 2)

#standardize the input variables between 0 and 1
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(xTrain)

xTrain = scaler.transform(xTrain)
xVald = scaler.transform(xVald)

#K-NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(xTrain, yTrain)

from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(xTrain, yTrain)
print (accuracy_score(yTrain, knn.predict(xTrain)))
print (accuracy_score(yVald, knn.predict(xVald)))

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(xTrain, yTrain)
print (accuracy_score(yTrain, knn.predict(xTrain)))
print (accuracy_score(yVald, knn.predict(xVald)))

knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(xTrain, yTrain)
print (accuracy_score(yTrain, knn.predict(xTrain)))
print (accuracy_score(yVald, knn.predict(xVald)))

n_range = np.arange(1, 21)
n_range

n_score = pd.DataFrame(columns=['NumNeig', 'TrainScore', 'ValdScore'])
n_score

for i in n_range:
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(xTrain, yTrain)
    accTrain = accuracy_score(yTrain, knn.predict(xTrain))
    accVald = accuracy_score(yVald, knn.predict(xVald))
    n_score = n_score.append({'NumNeig': i, 'TrainScore':accTrain, 'ValdScore':accVald}, ignore_index=True)

n_score.head()

bestValdScore = n_score['ValdScore'].max()
bestValdScore

plt.figure(figsize=(12,8))
plt.plot(n_score['NumNeig'], n_score['TrainScore'], label = 'Training')
plt.plot(n_score['NumNeig'], n_score['ValdScore'], label = 'Validation', c='red')
plt.axhline(bestValdScore, c='black', linestyle = ':')
plt.title('Accuracy Score for Training and Validation Dataset')
plt.ylabel('Accuracy')
plt.xlabel('Number of Neighbors Used')
plt.legend()
plt.show()

##Classification Tree
from sklearn.tree import DecisionTreeClassifier

decTree = DecisionTreeClassifier(max_leaf_nodes=19)

decTree.fit(xTrain, yTrain)

import graphviz
from sklearn import tree

dot_data = tree.export_graphviz(decTree, out_file=None)
graph = graphviz.Source(dot_data)
graph

from sklearn.metrics import accuracy_score

accuracy_score(yVald, decTree.predict(xVald))

leaf_range=np.arange(2,40)
leaf_range

leaf_score = pd.DataFrame(columns=['NofLeaves', 'TrainScore', 'ValidationScore'])
leaf_score

for i in leaf_range:
    decTree= DecisionTreeClassifier(max_leaf_nodes=i)
    decTree.fit(xTrain, yTrain)
    accTrain = accuracy_score(yTrain, decTree.predict(xTrain))
    accVald= accuracy_score(yVald, decTree.predict(xVald))
    leaf_score = leaf_score.append({'NofLeaves': i, 'TrainScore': accTrain, 'ValidationScore':accVald}, ignore_index=True)

leaf_score.head()

leaf_score

bestValdScore = leaf_score['ValidationScore'].max()

plt.figure(figsize=(12,8))
plt.plot(leaf_score['NofLeaves'], leaf_score['TrainScore'], label='Trainning')
plt.plot(leaf_score['NofLeaves'], leaf_score['ValidationScore'], label='Validation', color='red')
plt.title('Accuracy Score for Training and Validation Dataset')
plt.ylabel('Accuracy')
plt.xlabel('Number of leaf Used')
plt.axhline(bestValdScore, color = 'black', linestyle=':')
plt.legend()
plt.show()

from sklearn.neural_network import MLPClassifier

nn = MLPClassifier(hidden_layer_sizes=(50,50,50,), activation='relu', max_iter=200, solver='lbfgs')

nn.fit(xTrain, yTrain)

print(accuracy_score (yVald, nn.predict(xVald)))
print(accuracy_score (yTrain, nn.predict(xTrain)))
nn.score(xTrain, yTrain)

nn.predict(xTrain).sum()

print(Bankdata['month'].value_counts())
cross_month = pd.crosstab(Bankdata['month'], Bankdata['term_deposit']).apply(lambda x: x/x.sum() * 100)
cross_month

# Let's create a date column that will be interesting.
# We will assume the year is 2017
Bankdata['year'] = 2017
lst = [Bankdata]

# Create a column with the numeric values of the months.
for column in lst:
    column.loc[column["month"] == "jan", "month_int"] = 1
    column.loc[column["month"] == "feb", "month_int"] = 2
    column.loc[column["month"] == "mar", "month_int"] = 3
    column.loc[column["month"] == "apr", "month_int"] = 4
    column.loc[column["month"] == "may", "month_int"] = 5
    column.loc[column["month"] == "jun", "month_int"] = 6
    column.loc[column["month"] == "jul", "month_int"] = 7
    column.loc[column["month"] == "aug", "month_int"] = 8
    column.loc[column["month"] == "sep", "month_int"] = 9
    column.loc[column["month"] == "oct", "month_int"] = 10
    column.loc[column["month"] == "nov", "month_int"] = 11
    column.loc[column["month"] == "dec", "month_int"] = 12

# Change datatype from int32 to int64
Bankdata["month_int"] = Bankdata["month_int"].astype(np.int64)

f,ax=plt.subplots(1,2,figsize=(18,8))
colors=["#F08090", "#00FA9A"]
labels = 'Refused T.D', 'Accepted a T.D'
Bankdata['term_deposit'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True, colors=colors, labels=labels,fontsize=14)
ax[0].set_title('Term Deposits', fontsize=20)
sns.countplot('term_deposit',data=Bankdata,ax=ax[1], palette=colors)
ax[1].set_title('Term Deposits', fontsize=20)
ax[1].set_xticklabels(['T.D Refused', 'T.D Accepted'], fontsize=14)
plt.show()

fig = plt.figure(figsize=(10,4),)
ax=sns.kdeplot(Bankdata.loc[(Bankdata['term_deposit'] == 0),'month_int'] , color='#F08080',shade=True,label='Refused T.D')
ax=sns.kdeplot(Bankdata.loc[(Bankdata['term_deposit'] == 1),'month_int'] , color='#00FA9A',shade=True, label='Accepted T.D')
ax.set(xlabel='Months', ylabel='Frequency')
plt.title('Term Deposits by Month')

# Create the Season column.
Bankdata['season'] = np.nan
lst=[Bankdata]
# The conditions for determining each of the seasons.
for column in lst:
    column.loc[(column["month_int"] >= 3) & (column["month_int"] <= 5), 'season'] = 'spring'
    column.loc[(column["month_int"] >= 6) & (column["month_int"] <= 8), 'season'] = 'summer'
    column.loc[(column["month_int"] >= 9) & (column["month_int"] <= 11), 'season'] = 'fall'
    column.loc[column["month_int"] <= 2, 'season'] = 'winter'
    column.loc[column["month_int"] == 12, 'season'] = 'winter'
    
Bankdata['season'].value_counts()

fig = plt.figure(figsize=(15,4),)

colors = ['#F08080', '#00FA9A']

ax = sns.countplot(y='season', data=Bankdata,
           hue='term_deposit',
           palette = colors
          ) 

plt.title("Season-Wise Term Deposits", fontsize=16)
plt.ylabel("Seasons", fontsize=12)
plt.xlabel("Market Activity", fontsize=12)
legend_name = plt.legend()
legend_name.get_texts()[0].set_text('T.D. Refused')
legend_name.get_texts()[1].set_text('T.D. Accepted')

plt.show()

##Analysis of Term Deposit based on Age 
fig = plt.figure(figsize=(10,4),)
ax=sns.kdeplot(Bankdata.loc[(Bankdata['term_deposit'] == 0),'age'] , color='#bf40bf',shade=True,label='T.D. Refused')
ax=sns.kdeplot(Bankdata.loc[(Bankdata['term_deposit'] == 1),'age'] , color='#00FA9A',shade=True, label='T.D. Accepted')
ax.set(xlabel='Age of Individuals', ylabel='Frequency')
plt.title('Term Subscriptions by Age')

corr = Bankdata.corr()

sns.heatmap(corr,annot=True,cmap='RdYlGn',linewidths=0.2,annot_kws={'size':9})
fig=plt.gcf()
fig.set_size_inches(14,12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()


knn_output= knn.predict(xTrain)

decTree_output=decTree.predict(xTrain)

# Creating dataframe for NN, KNN, and DecTree
nn_output=pd.DataFrame(nn.predict(xTrain))

knn_output=pd.DataFrame(knn_output)


decTree_output=pd.DataFrame(decTree_output)


ensemble_Train=pd.concat([nn_output,decTree_output,knn_output],axis=1)

col1=(['nn_output','decTree_output','knn_output'])
ensemble_Train.columns=col1
ensemble_Train.head()

nn_ensemble= MLPClassifier(hidden_layer_sizes=(50,50,50,), activation='relu', max_iter=200, solver='lbfgs')
nn_ensemble.fit(ensemble_Train, yTrain)
print("Accuracy Score: ",accuracy_score(yTrain,nn_ensemble.predict(ensemble_Train)))